### Computation of TF-Glycogene interactions

#### Prepare single-cell gene expression data

In [1]:
#%% Load libraries
import os
import scanpy as sc
import pandas as pd
import numpy as np
import pickle
import json
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statannotations.Annotator import Annotator
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import FuncFormatter

#%% Define directory locations (Change according to folder structure preferred)
mainDir = '/Users/rudi/Data/TS_Codes/TF/' 
dataDir = '/Users/rudi/Data/TS_Codes/data/'
processedDir = ''.join([dataDir, 'processed/'])
figDir = ''.join([mainDir, 'figures/'])
sc.settings.figdir = figDir

os.chdir(mainDir)

#%% Set scanpy parameters
sc.set_figure_params(dpi=100, color_map = 'viridis_r')
sc.settings.n_jobs=2
sc.settings.verbosity = 1
sns.set_style("ticks", {'axes.grid' : False})


In [2]:
#%% Load Tabula data 
tabula_file = ''.join([processedDir, 'TS_norm_log1p.h5ad'])
tabula = sc.read_h5ad(tabula_file)

#%% Get decontaminated counts
tabula.X = tabula.layers['decontXcounts'].copy()

#%% Get scaled UMI counts
sc.pp.normalize_total(tabula, target_sum=1e4) # Normalize counts such that each cell has a total of 1e4


In [3]:
#%% Scripts for subsetting data

# Subset data based on sequencing technology(10X or Smart-Seq) 
tabula = tabula[tabula.obs['assay']=='10x 3\' v3'] #Use for 10X

#%% Subset data to specific genes (in this case protein-coding transcripts only)
genes_found = pd.read_csv(''.join([dataDir, 'Tabulagenes_ensemblmetadata.csv'])) # BioMart mapping of genes
types_of_genes = genes_found['gene_biotype'] #Gene biotype has info on which type of RNA each ensembl id is mapped to

# Display the counts of different transcript types
print('The types of genes found in dataset are:')
print(types_of_genes.value_counts())

# Merge ensembl id mapping with Tabula variables
tabula.var = (tabula.var.reset_index().
              merge(genes_found, how = 'left', left_on = 'gene_id', right_on = 'ensembl_gene_id').
              set_index('gene_id')) #Merge the info of mapping to adata object


The types of genes found in dataset are:
protein_coding                        19847
lncRNA                                14936
processed_pseudogene                  10114
unprocessed_pseudogene                 2567
misc_RNA                               2210
snRNA                                  1901
miRNA                                  1875
TEC                                    1048
snoRNA                                  941
transcribed_unprocessed_pseudogene      940
transcribed_processed_pseudogene        506
rRNA_pseudogene                         494
IG_V_pseudogene                         187
transcribed_unitary_pseudogene          150
IG_V_gene                               145
TR_V_gene                               106
unitary_pseudogene                       96
TR_J_gene                                79
scaRNA                                   49
rRNA                                     47
IG_D_gene                                37
TR_V_pseudogene                    

In [4]:
#%% Load GlycoEnzOnto gene sets and subset data to glycogenes
glycoOntoDir = ''.join([dataDir, 'GlycoEnzOnto/'])
GlycoEnzOntoFile = ''.join([glycoOntoDir, 'GlycoEnzOntoEns.json'])# Ensembl IDs of all glycogenes
with open(GlycoEnzOntoFile, 'r') as json_file:
    glycoSet = json.load(json_file)

GlycoEnzOntoFile = ''.join([glycoOntoDir, 'GlycoEnzOnto.json'])# Gene symbols of all glycogenes
with open(GlycoEnzOntoFile, 'r') as json_file:
    glycoSetnames = json.load(json_file)

# Filter glyco gene sets to gene symbols in data
glycoSetnames_filter = {k: list({e for i, e in enumerate(v) if e in tabula.var['feature_name'].tolist()}) 
            for k, v in glycoSetnames.items()}

# Filter glyco gene sets to ensembleID in data
glycoSet_filter = {k: list({e for i, e in enumerate(v) if e in tabula.var.index.tolist()}) 
            for k, v in glycoSet.items()}

# Get merged list of glycogenes detected in dataset and remove duplicates
glycoGeneNames = list({e for k, v in glycoSetnames_filter.items() for i, e in enumerate(v)}) 
glycoGenes = list({e for k, v in glycoSet_filter.items() for i, e in enumerate(v)})

# Check missing glycogenes in dataset
glycoGeneNames_notfound ={e for k, v in glycoSetnames.items() for i, e in enumerate(v)} - set(tabula.var['feature_name'].tolist())
glycoGenes_notfound ={e for k, v in glycoSet.items() for i, e in enumerate(v)} - set(tabula.var.index.tolist())

print('Glycogenes missing in dataset are:')
print(glycoGeneNames_notfound)

#Add additional column to data.var to indicate if genes are glycogenes
tabula.var['isglyco'] = tabula.var['feature_name'].isin(glycoGeneNames)


Glycogenes missing in dataset are:
{'GALNT19', 'UGT1A5', 'UGT1A3'}


In [5]:
# Load TF set
tf_file = ''.join([dataDir,'tf_set.pkl'])

with open(tf_file, 'rb') as f:
    tf_set = pickle.load(f)

#Add additional column to data.var to indicate if genes are TFs
tabula.var['isTF'] = tabula.var['feature_name'].isin(tf_set)


In [7]:
# Subset to TFs and Glycogenes
tabula = tabula[:, (tabula.var['isglyco'] == True) | (tabula.var['isTF'] == True)].copy()

#### Computation of mutual information between TF and glycogene

In [20]:
# Parallel calculation for MI
import csv
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from mi_computation import compute_mi

# Define the path for the output file
output_file_path = ''.join([mainDir, 'mi_results.txt'])

# Convert the expression matrix to a dense format if it's stored as a sparse matrix
expression = tabula.X.toarray() if scipy.sparse.issparse(tabula.X) else tabula.X

# Get the indices of glyco genes and TF genes
glycogenes = np.where(tabula.var['isglyco'])[0]
tfs = np.where(tabula.var['isTF'])[0]

# Use a process pool to parallelize the computation
with ProcessPoolExecutor(max_workers=8) as executor:
    # Prepare the list of tasks
    tasks = [(glycogene, tf) for glycogene in glycogenes for tf in tfs]

    # Open the output file in append mode
    with open(output_file_path, 'a', newline='') as f:
        writer = csv.writer(f, delimiter='\t')

        # Write the header if the file is empty (i.e., we're starting fresh)
        if f.tell() == 0:
            writer.writerow(['TF', 'Glycogene', 'MI'])

        # Submit the tasks to the pool and process completed tasks
        future_to_mi = {
            executor.submit(compute_mi, expression[:, pair[0]], expression[:, pair[1]]): pair for pair in tasks
        }

        for future in concurrent.futures.as_completed(future_to_mi):
            pair = future_to_mi[future]
            try:
                mi = future.result()
                glycogene_name = tabula.var['feature_name'][pair[0]]
                tf_name = tabula.var['feature_name'][pair[1]]

                # Write the result for this pair as soon as it's available
                writer.writerow([tf_name, glycogene_name, mi])
            except Exception as exc:
                print('%r generated an exception: %s' % (pair, exc))

#### Computation of Pearson and Spearman correlations between TF and glycogene

In [ ]:
#Parallel calculation for correlations
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from mi_computation import compute_pearson, compute_spearman

# Convert the expression matrix to a dense format if it's stored as a sparse matrix
expression = tabula.X.toarray() if scipy.sparse.issparse(tabula.X) else tabula.X

# Get the indices of glyco genes and TF genes
glycogenes = np.where(tabula.var['isglyco'])[0]
tfs = np.where(tabula.var['isTF'])[0]

# Prepare dictionaries to store the results
pearson_results = {}
spearman_results = {}

# Use a process pool to parallelize the computation
with ProcessPoolExecutor(max_workers=8) as executor:
    # Prepare the list of tasks for each type of computation
    tasks = [(glycogene, tf) for glycogene in glycogenes for tf in tfs]
    
    # Submit the tasks to the pool for Pearson correlation
    future_to_pearson = {
        executor.submit(compute_pearson, expression[:, pair[0]], expression[:, pair[1]]): pair for pair in tasks
    }

    # Submit the tasks to the pool for Spearman correlation
    future_to_spearman = {
        executor.submit(compute_spearman, expression[:, pair[0]], expression[:, pair[1]]): pair for pair in tasks
    }
    
    # Collect the results as they are completed for Pearson
    for future in concurrent.futures.as_completed(future_to_pearson):
        pair = future_to_pearson[future]
        try:
            pearson_corr = future.result()
            glycogene_name = tabula.var['feature_name'][pair[0]]
            tf_name = tabula.var['feature_name'][pair[1]]
            pearson_results.setdefault(glycogene_name, {})[tf_name] = pearson_corr
        except Exception as exc:
            print('%r generated an exception: %s' % (pair, exc))

    # Collect the results as they are completed for Spearman
    for future in concurrent.futures.as_completed(future_to_spearman):
        pair = future_to_spearman[future]
        try:
            spearman_corr = future.result()
            glycogene_name = tabula.var['feature_name'][pair[0]]
            tf_name = tabula.var['feature_name'][pair[1]]
            spearman_results.setdefault(glycogene_name, {})[tf_name] = spearman_corr
        except Exception as exc:
            print('%r generated an exception: %s' % (pair, exc))


In [9]:
# Save Pearson correlation results
pearson_list = []
for glycogene, tf_dict in pearson_results.items():
    for tf, pearson in tf_dict.items():
        pearson_list.append((tf, glycogene, pearson))

with open('pearson_results.txt', 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(['TF', 'Glycogene', 'score'])
    writer.writerows(pearson_list)

# Save Spearman correlation results
spearman_list = []
for glycogene, tf_dict in spearman_results.items():
    for tf, spearman in tf_dict.items():
        spearman_list.append((tf, glycogene, spearman))

with open('spearman_results.txt', 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(['TF', 'Glycogene', 'score'])
    writer.writerows(spearman_list)


#### Assesment of accuracy by AUROC and AUPR

In [35]:
# Evaluate AUROC and AUPR for MI
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, auc

# Load the TFLink.tsv file
TFfile = ''.join([dataDir,'TFLink.tsv'])
TFLink = pd.read_csv(TFfile, delimiter='\t')
TFLink = TFLink[['Name.TF', 'Name.Target']].rename(columns={'Name.TF': 'TF', 'Name.Target': 'Gene'})

# Load the mi_results.txt file
results = pd.read_csv('mi_results.txt', delimiter='\t')

# Get unique TFs and glycogenes from nmi
TFs = results['TF'].unique()
glycogenes = results['Glycogene'].unique()

# Subset the TFLink DataFrame
TFLink = TFLink[(TFLink['TF'].isin(TFs)) & (TFLink['Gene'].isin(glycogenes))]

# Merge the nmi DataFrame with the TFLink DataFrame to label each TF-Gene pair
results = results.merge(TFLink.assign(link=1), how='left', left_on=['TF', 'Glycogene'], right_on=['TF', 'Gene'])

# Fill missing values in the NMI column with 0 and in the link column with 0
results['MI'] = results['MI'].fillna(0)
results['link'] = results['link'].fillna(0)

# Use the NMI scores as the prediction scores and the link column as the true labels
scores = results['MI']
labels = results['link']

# Compute the AUROC
auroc = roc_auc_score(labels, scores)
print('AUROC for MI:', auroc)

# Generate the precision-recall curve
precision, recall, _ = precision_recall_curve(labels, scores)

# Calculate the AUPR
aupr = auc(recall, precision)
print('AUPR for MI:', aupr)

# Calculate the fraction of positive cases in the ground truth
num_positive_cases = labels.sum()
total_cases = len(labels)
fraction_positive = num_positive_cases / total_cases

print('Fraction of positive cases:', fraction_positive)

AUROC for MI: 0.6585762928081625
AUPR for MI: 0.4447855158386184
Fraction of positive cases: 0.3178737331081081


In [18]:
# Evaluate AUROC and AUPR
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, auc

# Load the TFLink.tsv file
TFfile = ''.join([dataDir,'TFLink.tsv'])
TFLink = pd.read_csv(TFfile, delimiter='\t')
TFLink = TFLink[['Name.TF', 'Name.Target']].rename(columns={'Name.TF': 'TF', 'Name.Target': 'Gene'})

# Load the mi_results.txt file
results = pd.read_csv('pearson_results.txt', delimiter='\t')

# Get unique TFs and glycogenes from nmi
TFs = results['TF'].unique()
glycogenes = results['Glycogene'].unique()

# Subset the TFLink DataFrame
TFLink = TFLink[(TFLink['TF'].isin(TFs)) & (TFLink['Gene'].isin(glycogenes))]

# Merge the nmi DataFrame with the TFLink DataFrame to label each TF-Gene pair
results = results.merge(TFLink.assign(link=1), how='left', left_on=['TF', 'Glycogene'], right_on=['TF', 'Gene'])

# Fill missing values in the NMI column with 0 and in the link column with 0
results['score'] = results['score'].fillna(0)
results['link'] = results['link'].fillna(0)

# Use the NMI scores as the prediction scores and the link column as the true labels
scores = results['score']
labels = results['link']

# Compute the AUROC
auroc = roc_auc_score(labels, scores)
print('AUROC for Pearson corr.:', auroc)

# Generate the precision-recall curve
precision, recall, _ = precision_recall_curve(labels, scores)

# Calculate the AUPR
aupr = auc(recall, precision)
print('AUPR for Pearson corr.:', aupr)

# Calculate the fraction of positive cases in the ground truth
num_positive_cases = labels.sum()
total_cases = len(labels)
fraction_positive = num_positive_cases / total_cases

print('Fraction of positive cases:', fraction_positive)

AUROC for Pearson corr.: 0.5475275436733957
AUPR for Pearson corr.: 0.36724340612588785
Fraction of positive cases: 0.3178737331081081


In [19]:
# Evaluate AUROC and AUPR
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, auc

# Load the TFLink.tsv file
TFfile = ''.join([dataDir,'TFLink.tsv'])
TFLink = pd.read_csv(TFfile, delimiter='\t')
TFLink = TFLink[['Name.TF', 'Name.Target']].rename(columns={'Name.TF': 'TF', 'Name.Target': 'Gene'})

# Load the mi_results.txt file
results = pd.read_csv('spearman_results.txt', delimiter='\t')

# Get unique TFs and glycogenes from nmi
TFs = results['TF'].unique()
glycogenes = results['Glycogene'].unique()

# Subset the TFLink DataFrame
TFLink = TFLink[(TFLink['TF'].isin(TFs)) & (TFLink['Gene'].isin(glycogenes))]

# Merge the nmi DataFrame with the TFLink DataFrame to label each TF-Gene pair
results = results.merge(TFLink.assign(link=1), how='left', left_on=['TF', 'Glycogene'], right_on=['TF', 'Gene'])

# Fill missing values in the NMI column with 0 and in the link column with 0
results['score'] = results['score'].fillna(0)
results['link'] = results['link'].fillna(0)

# Use the NMI scores as the prediction scores and the link column as the true labels
scores = results['score']
labels = results['link']

# Compute the AUROC
auroc = roc_auc_score(labels, scores)
print('AUROC for Spearman corr.:', auroc)

# Generate the precision-recall curve
precision, recall, _ = precision_recall_curve(labels, scores)

# Calculate the AUPR
aupr = auc(recall, precision)
print('AUPR for Spearman corr.:', aupr)

# Calculate the fraction of positive cases in the ground truth
num_positive_cases = labels.sum()
total_cases = len(labels)
fraction_positive = num_positive_cases / total_cases

print('Fraction of positive cases:', fraction_positive)

AUROC for Spearman corr.: 0.6026565985676249
AUPR for Spearman corr.: 0.3963190302482294
Fraction of positive cases: 0.3178737331081081
